In [27]:
import requests
from bs4 import BeautifulSoup

In [3]:
#tianjianmeng123

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import matplotlib.pyplot as plt
from nltk import FreqDist
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import re
import nltk
import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import networkx as nx
from transformers import BartTokenizer, BartForConditionalGeneration

In [4]:
url = []
url.append("https://eps.leeds.ac.uk/computing/staff/8807/dr-noorhan-abbas")
url.append("https://eps.leeds.ac.uk/mechanical-engineering/staff/12626/dr-ban-al-jassani")
url.append("https://eps.leeds.ac.uk/computing/staff/11465/dr-sharib-ali")
url.append("https://eps.leeds.ac.uk/computing/staff/635/dr-mohammad-ammar-alsalka")
url.append("https://eps.leeds.ac.uk/computing/staff/8784/dr-abdulrahman-altahhan")
url.append("https://eps.leeds.ac.uk/computing/staff/33/professor-eric-atwell")
url.append("https://eps.leeds.ac.uk/computing/staff/12946/dr-zeeshan")
url.append("https://eps.leeds.ac.uk/computing/staff/10151/dr-fatemeh-banaie")
url.append("https://eps.leeds.ac.uk/computing/staff/523/dr-peter-bollada")
url.append("https://eps.leeds.ac.uk/computing/staff/115/professor-andy-bulpitt")
url.append("https://eps.leeds.ac.uk/computing/staff/11489/martin-callaghan")
url.append("https://eps.leeds.ac.uk/computing/staff/499/professor-hamish-carr")
url.append("https://eps.leeds.ac.uk/computing/staff/285/dr-natasha-shakhlevich")
url.append("https://eps.leeds.ac.uk/computing/staff/12765/omar-shaur-choudhry")

df = pd.DataFrame(columns=['Name', 'text'])
df.loc[0] = ['Noorhan Abbas', '']
df.loc[1] = ['Ban Al-Jassani', '']
df.loc[2] = ['Sharib Ali', '']
df.loc[3] = ['Mohammad Ammar Alsalka', '']
df.loc[4] = ['Abdulrahman Altahhan', '']
df.loc[5] = ['Eric Atwell', '']
df.loc[6] = ['Zeeshan', '']
df.loc[7] = ['Fatemeh Banaie', '']
df.loc[8] = ['Peter Bollada', '']
df.loc[9] = ['Andy Bulpitt', '']
df.loc[10] = ['Martin Callaghan', '']
df.loc[11] = ['Hamish Carr', '']
df.loc[12] = ['Natasha Shakhlevich', '']
df.loc[13] = ['Omar Shaur Choudhry', '']

count = 0
for i in url:
    response = requests.get(i)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'html.parser')
    cms = soup.find('div', class_='cms')
    profile = cms.text

    df.loc[count, 'text'] = profile
    count = count + 1

In [29]:
def preprocess_text(text):
    """
    对文本进行预处理
    """
    text = text.lower()  # 转换为小写字母
    text = text.replace("'", "")  # 去除单引号
    text = text.replace("-", "")  # 去除破折号
    text = "".join([char for char in text if char.isalpha() or char.isspace()])  # 保留字母和空格
    return text

def tokenize_text(text):
    """
    将文本转换为tokens列表
    """
    tokens = text.split()
    return tokens

def plot_most_frequent_words(tokens, title):
    """
    绘制文本中出现频率最高的词汇柱状图
    """
    fdist = FreqDist(tokens)
    words_df = pd.DataFrame({'word': list(fdist.keys()), 'count': list(fdist.values())})
    # 选择出现频率最高的前20个词汇
    d = words_df.nlargest(columns="count", n=10)
    plt.figure(figsize=(16, 9))
    sns.barplot(data=d, x="word", y="count")
    plt.title(title)
    plt.show()


def plot_pearson_correlation_coefficient(X, data):
    """
    绘制各个文本之间的Pearson相关系数热力图
    """
    corr_matrix = np.zeros((len(data), len(data)))
    for i, j in itertools.combinations(range(len(data)), 2):
        corr, _ = pearsonr(X.toarray()[i], X.toarray()[j])
        corr_matrix[i, j] = corr
        corr_matrix[j, i] = corr
    tick_labels = data['title']
    sns.set(style="white")
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    f, ax = plt.subplots(figsize=(10, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
                xticklabels=tick_labels, yticklabels=tick_labels,
                annot=True, fmt='.2f', annot_kws={'size': 10})
    plt.title("Pearson Correlation Coefficient Matrix")
    plt.show()


def recommend_courses(sinput, df):
    # 引入CountVectorizer库，用于文本向量化
    vectorizer = CountVectorizer(tokenizer=tokenize_text, preprocessor=preprocess_text)

    # 将data中的文本转换成向量X，每行代表一个课程的向量表示
    X = vectorizer.fit_transform(df['text'])

    # 将用户输入的文本转换成向量表示
    student_input = vectorizer.transform(sinput.split())

    # 计算用户输入的文本和所有课程向量之间的余弦相似度
    cosine_sim = cosine_similarity(X, student_input)

    # 为每个课程在data中增加一列'similarity'，表示该课程与用户输入的文本的相似度得分
    df['similarity'] = cosine_sim.sum(1)

    # 根据'similarity'得分进行降序排列，选择前3个得分最高
    top_courses = df.sort_values('similarity', ascending=False).head(3)['Name']
    df = df.sort_values('similarity', ascending=False)
    #return top_courses.values
    return df

In [30]:
df['text'] = df['text'].apply(lambda x: re.sub(r'<.*?>', '', x)) # Remove HTML tags
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # Remove punctuation
df['text'] = df['text'].apply(lambda x: x.lower()) # Convert to lowercase
# Tokenize data
nltk.download('punkt')
df['tokens'] = df['text'].apply(lambda x: nltk.word_tokenize(x))

# Remove stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Lemmatize data
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tianjianmeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tianjianmeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tianjianmeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
recommend_courses('I like deep learning', df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.444047,i am actively researching in the areas of deep...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.385935,i obtained my phd in image analysis awarded in...
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.368204,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.264764,profile profile profileprofile profileprofilep...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.251773,profileescriptionpreviously a research softwar...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.230658,profilevertisementi am currently a research fe...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.225647,hegemonyprofileescriptioni have earned a phd d...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.174870,i teach and research in artificial intelligenc...
9,Andy Bulpitt,\nresponsibilities\n\nhead of school\n\nresear...,"[responsibility, head, school, research, inter...",0.055132,suddenlyresponsibilitiespotionpotionpotionshie...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.027677,i am part of the modelling contingent of the l...


In [35]:
recommend_courses('I like playing piano', df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.334731,i teach and research in artificial intelligenc...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.323688,i obtained my phd in image analysis awarded in...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.230658,profilevertisementi am currently a research fe...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.226941,profile profile profileprofile profileprofilep...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.180517,hegemonyprofileescriptioni have earned a phd d...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.142430,i am actively researching in the areas of deep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.112096,i teach and research in artificial intelligenc...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.107903,profileescriptionpreviously a research softwar...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.027677,i am part of the modelling contingent of the l...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.000000,corridorprofileescriptionsenior programmer wit...


In [36]:
recommend_courses("I like data mining and text analysis", df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.937247,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.907763,profile profile profileprofile profileprofilep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.892287,i teach and research in artificial intelligenc...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.846568,i obtained my phd in image analysis awarded in...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.661882,i am actively researching in the areas of deep...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.611448,profileescriptionpreviously a research softwar...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.593944,suddenlyresponsibilitiesacebookacebookacebooks...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.593120,profilevertisementi am currently a research fe...
12,Natasha Shakhlevich,\nresponsibilities\n\nexaminations officer\n\n...,"[responsibility, examination, officer, researc...",0.474342,heresponsibilities are in the area of determin...
11,Hamish Carr,\n \n \n \n ...,"[research, project, research, project, im, cur...",0.424264,will Â d ________________________________...


In [53]:
recommend_courses("text analytics, data mining", df)

/Users/tianjianmeng/opt/anaconda3/envs/finalP/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Name,text,tokens,similarity,summary
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.273515,i teach and research in artificial intelligenc...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.269975,suddenlyresponsibilitiesacebookacebookacebooks...
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.267785,i teach and research in artificial intelligenc...
13,Omar Shaur Choudhry,\nprofile\nthirdyear computer science undergra...,"[profile, thirdyear, computer, science, underg...",0.113470,profile profile profileprofile profileprofilep...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.108917,i am actively researching in the areas of deep...
11,Hamish Carr,\n \n \n \n ...,"[research, project, research, project, im, cur...",0.070711,will Â d ________________________________...
10,Martin Callaghan,\nprofile\npreviously a research software engi...,"[profile, previously, research, software, engi...",0.035968,profileescriptionpreviously a research softwar...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.032951,profilevertisementi am currently a research fe...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.000000,corridorprofileescriptionsenior programmer wit...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.000000,i obtained my phd in image analysis awarded in...



Supervisors which have more contents will get more recommendations.  

Supervisors with short contents have a limited number of keywords and do not easily get recommendations.  

There are some supervisors who often get to be at the top of the recommendation table because their contents are so long and therefore the number of keywords is also bigger than others, and no matter what you enter, they always take the top spot in the recommendations.  

(Sharib Ali and Noorhan Abbas always gets high similarity scores.)

Text generation using Bart model

In [37]:
device = torch.device("mps:0")

In [38]:
# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
modelB = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Function to generate summaries
def generate_summary_brat(text):
    inputs = tokenizer.encode_plus(text, truncation=True, max_length=1024, return_tensors='pt')
    input_ids = inputs['input_ids'].to(modelB.device)
    attention_mask = inputs['attention_mask'].to(modelB.device)
    summary_ids = modelB.generate(input_ids, attention_mask=attention_mask, num_beams=8, max_length=100, early_stopping=False) # max_length=100 is original set
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [39]:
df['summary'] = df['text'].apply(generate_summary_brat)

In [40]:
df

,Name,text,tokens,similarity,summary
0,Noorhan Abbas,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.937247,i teach and research in artificial intelligenc...
1,Ban Al-Jassani,\nprofile\nsenior programmer with valuable exp...,"[profile, senior, programmer, valuable, experi...",0.293119,corridorprofileescriptionsenior programmer wit...
2,Sharib Ali,\nprofile\ni obtained my phd in image analysis...,"[profile, obtained, phd, image, analysis, awar...",0.846568,i obtained my phd in image analysis awarded in...
3,Mohammad Ammar Alsalka,\nresponsibilities\n\nmsc projects coordinator...,"[responsibility, msc, project, coordinator, re...",0.593944,suddenlyresponsibilitiesacebookacebookacebooks...
4,Abdulrahman Altahhan,\nprofile\ni have extesive experience in devel...,"[profile, extesive, experience, developing, le...",0.661882,i am actively researching in the areas of deep...
5,Eric Atwell,\nprofile\ni teach and research in artificial ...,"[profile, teach, research, artificial, intelli...",0.892287,i teach and research in artificial intelligenc...
6,Zeeshan,\nprofile\ni have earned a phd degree in robot...,"[profile, earned, phd, degree, robotics, auton...",0.406164,hegemonyprofileescriptioni have earned a phd d...
7,Fatemeh Banaie,\nprofile\ni am currently a research fellow in...,"[profile, currently, research, fellow, compute...",0.593120,profilevertisementi am currently a research fe...
8,Peter Bollada,\nprofile\napplied maths and physics degree op...,"[profile, applied, math, physic, degree, open,...",0.290604,i am part of the modelling contingent of the l...
9,Andy Bulpitt,\nresponsibilities\n\nhead of school\n\nresear...,"[responsibility, head, school, research, inter...",0.385922,suddenlyresponsibilitiespotionpotionpotionshie...


In [41]:
df.loc[0, 'summary']

'i teach and research in artificial intelligence python for data science data mining and text analytics in the school of computing at university of leeds\xa0 i\xa0worked in the edubots project which is an erasmus knowledge alliance project that aims to\xa0improve results and raise\xa0attainment levels\xa0in european higher education institutions this\xa0project focussed on exploring best practices and innovative use of chatbots and creating a learning community of educators\xa0i have experience in organising conferences and workshops\xa0'

TextRank generation

In [42]:
# Dr Abdulrahman Altahhan

x ="""I have extesive experience in developing, leading and managing postgraduate and undergraduate computer science HE programmes, particularly in the areas of AI and Data Science.

I am actively researching in the areas of Deep Reinforcement Learning, Neural Network and Machine Learning with hands-on applications and publications in image processing, autonomous vehicles and robotics. Autonomous learning from data is second nature to me; particularly I enjoy developing reinforcement learning algorithms, several of which have both a highly practical and theoretical potential.

I have developed a tool that was adopted throughout a university for quantifying and accurately modeling the data from modules and course outcomes, these algorithms and research have a high potential for commercialisation.

I have led a small research group in AI and Robotics along with a few programmes in Data Science, Intelligent Systems and Information Technology for both the undergraduate and graduate levels. I have taught numerous modules, the majority of which in AI and Machine Learning.

My main research area is Deep Reinforcement Learning applied to Intelligent Agents. I have developed several novel RL and DRL algorithms that seamlessly integrate experience replay within the learning process of the agent. I have applied these on games, stochastic processes and on predicting control signals for a prosthetic limp. I have developed several robot navigation models that employed deep reinforcement learning to visually allow a robot to learn to navigate in an unknown environment without prior experience.

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Student education
I have developed material for several modules for the new MSc. in Artificial Intelligence ODL programme. I lead modules such as Data Science and Reinforcement Learning on Robotics. I have also led modules such as Machine Learning, Neural Networks and Big Data Analysis.

I have served as a Programme Director for MSc. Data Science and Computational Intelligence, where I enjoy setting up suitable strategies for this programme to be closely related to both the industry and research (my favourite balance), nurturing it into popularity and growth.

I have obtained managerial skills by undertaking different leading roles and responsibilities, ex. appointed as acting dean of my college and coordinating BSc. in Business Information Technology program for three years. 

My experience of educating professional students includes proficiency in teaching several AI and computer science topics in multicultural settings. Extensively proposed and championed several curriculums in Data Science and AI at the Master and Undergraduate levels. 

Postgraduate research opportunities
We welcome enquiries from motivated and qualified applicants from all around the world who are interested in PhD study. Our research opportunities allow you to search for projects and scholarships."""

def generate_summary_textRank(x):
    model = Word2Vec(
    sentences= [word_tokenize(x)],
        vector_size=100, window=5, min_count=5, workers=2
    )

    sentences = sent_tokenize(x)
    sentences_feat = [
        np.array([model.wv[x] for x in word_tokenize(sentence.lower()) if x in model.wv]).mean(0)
        for sentence in sentences]

    sentences_feat = np.array(sentences_feat)

    from sklearn.metrics.pairwise import cosine_similarity
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentences_feat[i].reshape(1,100), sentences_feat[j].reshape(1,100))[0,0]
                

    nx_graph = nx.from_numpy_array(sim_mat)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            nx_graph[2][3]['weight'] = sim_mat[i][j]
    scores = nx.pagerank(nx_graph)

    return sentences[np.argmax(list(scores.values()))]

In [43]:
generate_summary_textRank(x)

'I have led a small research group in AI and Robotics along with a few programmes in Data Science, Intelligent Systems and Information Technology for both the undergraduate and graduate levels.'

Comparison with Bart

In [44]:
generate_summary_brat(x)

'I have extesive experience in developing, leading and managing postgraduate and undergraduate computer science HE programmes, particularly in the areas of AI and Data Science. I have also led a small research group in AI and Robotics. I am actively researching in the area of Deep Reinforcement Learning, Neural Network and Machine Learning with hands-on applications and publications in image processing, autonomous vehicles and robotics. Autonomous learning from data is second nature to me; particularly I enjoy developing reinforcement learning algorithms,'

In [45]:
# Omar Shaur Choudhry

y = """Profile
Third-year computer science undergraduate student and teaching assistant at the University of Leeds. I am the founder and director of my own company OC Solutions Ltd, providing bespoke information technology and software solutions and consultancy.

Responsibilities
Undergraduate Teaching Assistant
Research interests
I have completed my dissertation on Survival Analysis with Neural Networks for Chronic Heart Failure Patients.

Research projects
Any research projects I'm currently working on will be listed below. Our list of all research projects allows you to view and search the full list of projects in the faculty.

Professional memberships
Student Membership - BCS, The Chartered Institute for IT
Student Membership - IET, Institution of Engineering and Technology
Student education
In Semester 1, I mainly aided Level 1 students in lab sessions, focusing on the Introduction to Programming and Procedural Programming modules. I also helped Level 2 students in Web Application Development and Level 3 students in User Interfaces, Machine Learning, and Computer Graphics.

In Semester 2, I mainly assisting Level 2 students with the following modules: Data Mining, Networks, Artificial Intelligence, and Compilers Design & Construction. I also helped Level 1 students in Introduction to Web Technologies, Object-Oriented Programming, and Programming Project, and Level 3 students in Web Services & Web Data and Intelligent Systems and Robotics.""" #10/07

In [46]:
generate_summary_brat(y)

'Profile of me:Third-year computer science undergraduate student and teaching assistant at the University of Leeds. I am the founder and director of my own company OC Solutions Ltd, providing bespoke information technology and software solutions and consultancy.Profile of myself:Professionals:Computer Science, Computer Science, Engineering and Technology.Responsibilities:Undergraduate Teaching Assistant.Research interests:I have completed my dissertation on Survival Analysis with Neural Networks for Chronic Heart Failure Patients.Research projects:Research projects'

In [47]:
generate_summary_textRank(y)

'Professional memberships\nStudent Membership - BCS, The Chartered Institute for IT\nStudent Membership - IET, Institution of Engineering and Technology\nStudent education\nIn Semester 1, I mainly aided Level 1 students in lab sessions, focusing on the Introduction to Programming and Procedural Programming modules.'